# The Currencies data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/Currencies.ipynb)

In [1]:
library(DBI)
library(plutoDbR)
library(plutoR)
library(tidyverse)
options("scipen"=999)
source("/usr/share/pluto/config.R")

#initialize
currencies <- Currencies()

### get traded futures pairs

In [2]:
currencies$NseFuturesTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP)) %>%
  arrange(START_DT) %>%
  print()

# Source:     lazy query [?? x 3]
# Database:   NORWAY:StockViz:R
# Ordered by: START_DT
  SYMBOL START_DT   END_DT    
  <chr>  <chr>      <chr>     
1 GBPINR 2010-10-29 2019-09-13
2 USDINR 2010-10-29 2019-09-13
3 JPYINR 2010-10-29 2019-09-13
4 EURINR 2010-10-29 2019-09-13
5 USDJPY 2018-02-27 2019-09-13
6 GBPUSD 2018-02-27 2019-09-13
7 EURUSD 2018-02-27 2019-09-13


### get the latest USDINR option chain for the nearest expiry

In [3]:
maxDt <- (currencies$NseOptionsTimeSeries() %>%
            summarize(MAX_DT = max(TIME_STAMP)) %>%
            collect())$MAX_DT[[1]]

expiryDt <- (currencies$NseOptionsTimeSeries() %>%
            filter(TIME_STAMP == maxDt & SYMBOL == 'USDINR') %>%
            summarize(MAX_DT = max(EXPIRY)) %>%
            collect())$MAX_DT[[1]]

currencies$NseOptionsTimeSeries() %>%
  filter(TIME_STAMP == maxDt & SYMBOL == 'USDINR' & EXPIRY == expiryDt) %>%
  arrange(STRIKE, OTYPE) %>%
  print()

# Source:     lazy query [?? x 13]
# Database:   NORWAY:StockViz:R
# Ordered by: STRIKE, OTYPE
   TIME_STAMP SYMBOL EXPIRY STRIKE OTYPE  HIGH   LOW  OPEN  CLOSE SETTLE
   <chr>      <chr>  <chr>   <dbl> <chr> <int> <int> <int>  <dbl>  <dbl>
 1 2019-09-13 USDINR 2020-…   65.5 CE        0     0     0 6.89    7.90 
 2 2019-09-13 USDINR 2020-…   65.5 PE        0     0     0 0.0725  0.218
 3 2019-09-13 USDINR 2020-…   65.8 CE        0     0     0 6.66    7.68 
 4 2019-09-13 USDINR 2020-…   65.8 PE        0     0     0 0.0825  0.24 
 5 2019-09-13 USDINR 2020-…   66   CE        0     0     0 6.44    7.46 
 6 2019-09-13 USDINR 2020-…   66   PE        0     0     0 0.0975  0.265
 7 2019-09-13 USDINR 2020-…   66.2 CE        0     0     0 6.22    7.24 
 8 2019-09-13 USDINR 2020-…   66.2 PE        0     0     0 0.112   0.292
 9 2019-09-13 USDINR 2020-…   66.5 CE        0     0     0 5.99    7.03 
10 2019-09-13 USDINR 2020-…   66.5 PE        0     0     0 0.13    0.322
# … with more rows, and 3 mor

### get the currencies tracked by AlphaVantage end-of-day

In [4]:
currencies$AvEodTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP)) %>%
  arrange(START_DT) %>%
  print(n=Inf)

# Source:     lazy query [?? x 3]
# Database:   WINDOWS:StockVizBeka:R
# Ordered by: START_DT
    SYMBOL START_DT   END_DT    
    <chr>  <chr>      <chr>     
  1 MYR    1998-01-23 2019-09-13
  2 TWD    2000-04-24 2019-09-13
  3 JPY    2000-04-27 2019-09-13
  4 CHF    2000-07-19 2019-09-13
  5 CAD    2000-07-27 2019-09-13
  6 HKD    2000-08-04 2019-09-13
  7 SGD    2000-08-10 2019-09-13
  8 CNY    2005-11-09 2019-09-13
  9 MOP    2011-04-07 2019-09-13
 10 BHD    2014-11-07 2019-09-13
 11 BIF    2014-11-07 2019-09-13
 12 BND    2014-11-07 2019-09-13
 13 BOB    2014-11-07 2019-09-13
 14 BRL    2014-11-07 2019-09-13
 15 BSD    2014-11-07 2019-09-13
 16 BWP    2014-11-07 2019-09-12
 17 BZD    2014-11-07 2019-09-13
 18 CDF    2014-11-07 2019-09-13
 19 CLF    2014-11-07 2019-09-12
 20 CLP    2014-11-07 2019-09-12
 21 CNH    2014-11-07 2019-09-13
 22 ZWL    2014-11-07 2015-10-15
 23 AFN    2014-11-07 2019-09-13
 24 ALL    2014-11-07 2019-09-12
 25 ANG    2014-11-07 2019-09-13
 26 ARS    2014

### get the currencies tracked by AlphaVantage 30-min bars 

In [5]:
currencies$Av30minTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP), COUNT = n()) %>%
  arrange(START_DT) %>%
  print(n=Inf)

# Source:     lazy query [?? x 4]
# Database:   WINDOWS:StockVizBeka:R
# Ordered by: START_DT
    SYMBOL START_DT            END_DT              COUNT
    <chr>  <chr>               <chr>               <int>
  1 CNY    2018-08-03 02:00:00 2019-09-13 08:30:00  6724
  2 AFN    2018-08-21 16:00:00 2019-09-13 22:00:00 16868
  3 AED    2018-08-21 16:00:00 2019-09-13 22:00:00 16889
  4 DZD    2018-08-21 16:00:00 2019-09-13 22:00:00 15583
  5 EGP    2018-08-21 16:00:00 2019-09-13 22:00:00 15612
  6 ERN    2018-08-21 16:00:00 2019-09-13 22:00:00 15597
  7 GHS    2018-08-21 16:00:00 2019-09-13 22:00:00 15582
  8 GMD    2018-08-21 16:00:00 2019-09-13 13:00:00 12148
  9 GNF    2018-08-21 16:00:00 2019-09-13 22:00:00 15596
 10 GTQ    2018-08-21 16:00:00 2019-09-13 22:00:00 15595
 11 GYD    2018-08-21 16:00:00 2019-09-13 22:00:00 15597
 12 HNL    2018-08-21 16:00:00 2019-09-13 22:00:00 15595
 13 HTG    2018-08-21 16:00:00 2019-09-13 22:00:00 15596
 14 IDR    2018-08-21 16:00:00 2019-09-13 22:00:00 

This notebook was created using [pluto](https://pluto.studio). Thank you for playing fair!